In [52]:
import torch

from pinder.core import get_index, PinderSystem
from pinder.core import PinderLoader
from pinder.core.loader import filters
from pinder.core.index.system import PinderSystem
from pinder.core import get_pinder_location
import re
import pandas as pd
import csv
import os
import random

UNIPROT_IDS_PATH="/scicore/home/schwede/pudziu0000/projects/gLM/data/PINDER/uniprot_ids_1024_512.lst"
UNIPROT_LINEAGES_PATH="/scicore/home/schwede/pudziu0000/projects/gLM/data/PINDER/uniprot_lineages_annotations_1024_512.tsv"
SPLITS_DIR="/scicore/home/schwede/pudziu0000/projects/gLM/data/PINDER/eubacteria_5_1024_512/"
SEED=42

random.seed(SEED)

def select_ids(max_length=1024, max_length_per_monomer=400):
    base_filters = [
        filters.FilterByMissingHolo(),
        filters.FilterSubByContacts(min_contacts=5, radius=10.0, calpha_only=True),
        filters.FilterDetachedHolo(radius=12, max_components=2),
    ]
    sub_filters = [
        filters.FilterSubByAtomTypes(min_atom_types=4),
        filters.FilterByHoloOverlap(min_overlap=5),
        filters.FilterByHoloSeqIdentity(min_sequence_identity=0.8),
        filters.FilterSubRmsds(rmsd_cutoff=7.5),
        filters.FilterDetachedSub(radius=12, max_components=2),
    ]
    split_pinder_ids = {}
    split_uniprot_ids = {}
    train_index = None
    for split in ["train", "val", "test"]:
        loader = PinderLoader(
            split=split,
            monomer_priority="holo",
            base_filters=base_filters,
            sub_filters=sub_filters,
        )
        print(loader)
        index = loader.index.merge(loader.metadata, on="id", how="left")
        index["length"] = index["length1"] + index["length2"]
        index["resolution"] = index["resolution"].astype("float32")
        index = index[
            (index["length"] + 2 <= max_length)
            & (index["length1"] <= max_length_per_monomer)
            & (index["length2"] <= max_length_per_monomer)
            & (index["label"] == "BIO")
        ]
        if(split == "train"): train_index = index
        pinder_ids = list(index["id"])
        uniprot_ids = set(list(index["uniprot_R"]) + list(index["uniprot_L"]))
        split_pinder_ids[split] = pinder_ids
        split_uniprot_ids[split] = uniprot_ids
    return split_pinder_ids, split_uniprot_ids, train_index

def reduce_train_redundancy(ids, index, max_per_cluster=5):
    # ids - list of ids of the training set
    # index - index of the training set
    ids = list(
        set(
            index.sort_values(
                ["resolution", "id"],
                ascending=[True, True],
            )
            .groupby("cluster_id", observed=False)
            .head(max_per_cluster)["id"]
        )
    )
    return ids

def make_random_pairs(L, pdb_L, chain_uniprot_L, uniprot_L, 
                      pdb_R, uniprot_R,
                      interacting_partners, GO, max_num_attempts):
    invalidity_flags = []
    # Shuffle until no same PDB pairs
    for attempt in range(max_num_attempts):  # Limit reattempts
        L_info = list(zip(L, pdb_L, chain_uniprot_L, uniprot_L))
        random.shuffle(L_info)
        L, pdb_L, chain_uniprot_L, uniprot_L = zip(*L_info)
        
        # If UniProt IDs of the new pair are known to contain interacting sequences, retry
        if any(uL in interacting_partners[uR] for uR, uL in zip(uniprot_R, uniprot_L)):
            if(attempt != max_num_attempts-1):
                continue
            else:
                invalidity_flags.append("interacting")
                
        # If UniProt IDs of the new pair have overlapping Molecular Function GOs, retry
        if any(bool(set(GO[uR]) & set(GO[uL])) for uR, uL in zip(uniprot_R, uniprot_L)):
            if(attempt != max_num_attempts-1):
                continue
            else:
                invalidity_flags.append("GO")

        # If PDB IDs of all pairs are non-matching, call it a success
        if all(pR != pL for pR, pL in zip(pdb_R, pdb_L)): break

    return L, pdb_L, chain_uniprot_L, uniprot_L, invalidity_flags


def remove_false_negatives(R, L, uniprot_R, uniprot_L, interacting_partners, 
                           GO, invalidity_flags, orig_pinder_ids):
    # If there are invalid pairings, execute removals
    if(invalidity_flags):
        to_remove = set()
        for i, (uR, uL) in enumerate(zip(uniprot_R, uniprot_L)):
            if "interacting" in invalidity_flags and uL in interacting_partners[uR]: to_remove.add(i)
            if "GO" in invalidity_flags and bool(set(GO[uR]) & set(GO[uL])): to_remove.add(i)
        
        valid_indices = set(range(len(R)))-to_remove
        R = [R[i] for i in valid_indices]
        L = [L[i] for i in valid_indices]
    
    # Forming new pinder_ids
    paired = zip(R, L)
    paired = ["--".join(pair) for pair in paired]
    
    # If new pinder_id is present in the original set, remove that record
    paired = [item for item in paired if item not in orig_pinder_ids]

    return paired

def make_negative_pairs(df, interacting_partners, GO, seed=SEED, max_num_attempts=100):    
    # Separate ids
    df[['R', 'L']] = df['pinder_id'].str.split('--', expand=True)
    df[['pdb_R', 'chain_uniprot_R']] = df['R'].str.split('__', expand=True)
    df[['chain_R', 'uniprot_R']] = df['chain_uniprot_R'].str.split('_', expand=True)
    df[['pdb_L', 'chain_uniprot_L']] = df['L'].str.split('__', expand=True)
    df[['chain_L', 'uniprot_L']] = df['chain_uniprot_L'].str.split('_', expand=True)
    
    # Group by phylum
    negative_ids = []
    negative_phyla = []
    orig_pinder_ids = set(df['pinder_id'])
    
    for phylum, group in df.groupby('phylum_R'):
        
        R = group['R'].tolist()
        L = group['L'].tolist()
        pdb_R = group['pdb_R'].tolist()
        pdb_L = group['pdb_L'].tolist()
        chain_uniprot_L = group['chain_uniprot_L'].tolist()
        uniprot_R = group['uniprot_R'].tolist()
        uniprot_L = group['uniprot_L'].tolist()

        L, pdb_L, chain_uniprot_L, uniprot_L, invalidity_flags = make_random_pairs(
            L, pdb_L, chain_uniprot_L, uniprot_L, pdb_R, uniprot_R, 
            interacting_partners, GO, max_num_attempts
        )
        
        # If there are invalid pairings, execute removals and get pairs
        paired = remove_false_negatives(R, L, uniprot_R, uniprot_L, 
            interacting_partners, GO, invalidity_flags, orig_pinder_ids
        )

        negative_ids.extend(paired)
        negative_phyla.extend([phylum]*len(paired))

    negative_records = list(zip(negative_ids, negative_phyla, negative_phyla))
    negative_df = pd.DataFrame(
        negative_records, 
        columns=['pinder_id', 'phylum_R', 'phylum_L']
    ).sample(frac=1, random_state=seed).reset_index(drop=True)
    
    return negative_df

    

In [53]:
os.environ["PINDER_BASE_DIR"] = "/scicore/home/schwede/durair0000/.local/share/"
get_pinder_location()

PosixPath('/scicore/home/schwede/durair0000/.local/share/pinder/2024-02')

In [54]:
pinder_ids, uniprot_ids, train_index = select_ids(max_length_per_monomer=512)

PinderLoader(split=train, monomers=holo, systems=1560682)
PinderLoader(split=val, monomers=holo, systems=1958)
PinderLoader(split=test, monomers=holo, systems=1955)


In [33]:
# Checking number of elements in each split
print(len(pinder_ids['train']),
      len(pinder_ids['val']),
      len(pinder_ids['test'])
)

print(len(uniprot_ids['train']),
      len(uniprot_ids['val']),
      len(uniprot_ids['test'])
)

675729 1799 1767
20907 1937 1906


In [34]:
# Selecting that that both defined UniProt IDs
pinder_ids['train'] = [row for row in pinder_ids['train'] if not re.search(r'UNDEFINED', row, re.IGNORECASE)]
pinder_ids['val'] = [row for row in pinder_ids['val'] if not re.search(r'UNDEFINED', row, re.IGNORECASE)]
pinder_ids['test'] = [row for row in pinder_ids['test'] if not re.search(r'UNDEFINED', row, re.IGNORECASE)]

uniprot_ids['train'].remove("UNDEFINED")
uniprot_ids['val'].remove("UNDEFINED")
uniprot_ids['test'].remove("UNDEFINED")

In [35]:
# Checking number of elements in each split after removal of "UNDEFINED"
print(len(pinder_ids['train']),
      len(pinder_ids['val']),
      len(pinder_ids['test'])
)

print(len(uniprot_ids['train']),
      len(uniprot_ids['val']),
      len(uniprot_ids['test'])
)

617197 1748 1688
20906 1936 1905


In [36]:
# Save UniProt IDs to a TXT file for "ID mapping"
with open(UNIPROT_IDS_PATH, 'w') as f:
    for split in ["train", "val", "test"]:
        for line in uniprot_ids[split]:
            f.write(f"{line}\n")

In [8]:
# Manually run the ID mapping on the UniProt

In [37]:
# Determine, which UniProt IDs are belonging to eubacteria with labelled phylum
lineages = pd.read_csv(UNIPROT_LINEAGES_PATH, sep='\t')

eubacteria_phyla = {}
eubacteria_annotated_interactions = {}
eubacteria_GO = {}

for index, row in lineages.iterrows():
    if isinstance(row['Taxonomic lineage (Ids)'], str) and "2 (superkingdom)" in row['Taxonomic lineage (Ids)']: 
        match = re.search(r'(\d+) \(phylum\)', row['Taxonomic lineage (Ids)'])
        if match:
            phylum_tax_id = match.group(1)
            eubacteria_phyla[row['From']] = phylum_tax_id
            
            # Retrieving UniProt IDs of the known interacting partners
            if(not pd.isna(row['Interacts with'])):
                eubacteria_annotated_interactions[row['From']] = [item.split('-')[0] for item in row['Interacts with'].split('; ')]
            else:
                eubacteria_annotated_interactions[row['From']] = []
            
            # Retrieving GO 
            if(not pd.isna(row['Gene Ontology (molecular function)'])):
                eubacteria_GO[row['From']] = re.findall(r"GO:\d+", row['Gene Ontology (molecular function)'])
            else:
                eubacteria_GO[row['From']] = []

In [38]:
# Determine, which PINDER IDs contain both bacterial proteins
eubacteria_pinder_ids = {'train': [], 'val': [], 'test': []}
eubacteria_pinder_phyla = {}

for split in ["train", "val", "test"]:
    for id in pinder_ids[split]:
        uniprot1 = id.split("--")[0].split("_")[-1]
        uniprot2 = id.split("--")[1].split("_")[-1]
        if(uniprot1 in eubacteria_phyla.keys() and uniprot2 in eubacteria_phyla.keys()):
            eubacteria_pinder_ids[split].append(id)
            eubacteria_pinder_phyla[id] = [eubacteria_phyla[uniprot1], eubacteria_phyla[uniprot2]]

In [39]:
print(len(eubacteria_pinder_ids['train']),
      len(eubacteria_pinder_ids['val']),
      len(eubacteria_pinder_ids['test'])
)

86221 906 885


In [40]:
# Remove redundancy from the training set
mask = train_index['id'].isin(eubacteria_pinder_ids['train'])
eubacteria_train_index = train_index[mask]
eubacteria_pinder_ids['train'] = reduce_train_redundancy(eubacteria_pinder_ids['train'], eubacteria_train_index, max_per_cluster=5)

In [41]:
# Number of elements in each split after redundancy removal
print(len(eubacteria_pinder_ids['train']),
      len(eubacteria_pinder_ids['val']),
      len(eubacteria_pinder_ids['test'])
)

8283 906 885


In [42]:
# Save IDs and phyla
print(f"Split\tClass\tNumber of instances")

for split in ["train", "val", "test"]:
    split_positives = []
    for id in eubacteria_pinder_ids[split]:
        record = {"pinder_id": id, "phylum_R": eubacteria_pinder_phyla[id][0], "phylum_L": eubacteria_pinder_phyla[id][1]}
        split_positives.append(record)

    split_positives = pd.DataFrame.from_dict(split_positives)
    split_negatives = make_negative_pairs(split_positives, eubacteria_annotated_interactions, eubacteria_GO)
    split_positives = split_positives[["pinder_id", "phylum_R", "phylum_L"]]

    # Balancing the set
    split_positives = split_positives.sample(n=len(split_negatives))

    # Labelling the set
    split_negatives['label'] = [0]*len(split_negatives)
    split_positives['label'] = [1]*len(split_positives)

    # Summary of the split
    print(f"{split}\t0\t{len(split_negatives)}\n{split}\t1\t{len(split_positives)}")

    # Merging the classes
    split_all = pd.concat([split_positives, split_negatives])
    
    # Shuffling
    split_all = split_all.sample(frac=1).reset_index(drop=True)

    # Saving
    split_all.to_csv(f"{SPLITS_DIR}/{split}.csv", index=False)

Split	Class	Number of instances
train	0	7670
train	1	7670
val	0	823
val	1	823
test	0	827
test	1	827


In [59]:
from Bio import SeqIO

fasta_sequences = SeqIO.parse("/scicore/home/schwede/pudziu0000/projects/gLM/data/PINDER/uniprot_seqs_1024_512.fasta",'fasta')

for i, record in enumerate(fasta_sequences):
    if(i < 10): 
        print(record.seq)


MIDKSAFVHPTAIVEEGASIGANAHIGPFCIVGPHVEIGEGTVLKSHVVVNGHTKIGRDNEIYQFASIGEVNQDLKYAGEPTRVEIGDRNRIRESVTIHRGTVQGGGLTKVGSDNLLMINAHIAHDCTVGNRCILANNATLAGHVSVDDFAIIGGMTAVHQFCIIGAHVMVGGCSGVAQDVPPYVIAQGNHATPFGVNIEGLKRRGFSREAITAIRNAYKLIYRSGKTLDEVKPEIAELAETYPEVKAFTDFFARSTRGLIR
MNRSVNIAKNLIQTYRAMSVQSRFAFSTREEEWLDKRTKSQEKVYFDQEDRKAMKRLLEKLNTTSKFVEDSEYLAPQNLEVENILKRYHINYTQALIDELVDWKTGKN
MARFRGSNWKKSRRLGISLSGTGKELEKRPYAPGQHGPNQRKKLSEYGLQLREKQKLRYLYGMTERQFRNTFDIAGKKFGVHGENFMILLASRLDAVVYSLGLARTRRQARQLVNHGHILVDGKRVDIPSYSVKPGQTISVREKSQKLNIIVESVEINNFVPEYLNFDADSLTGTFVRLPERSELPAEINEQLIVEYYSR
MALHVPKAPGFAQMLKEGAKHFSGLEEAVYRNIQACKELAQTTRTAYGPNGMNKMVINHLEKLFVTNDAATILRELEVQHPAAKMIVMASHMQEQEVGDGTNFVLVFAGALLELAEELLRIGLSVSEVIEGYEIACRKAHEILPNLVCCSAKNLRDIDEVSSLLRTSIMSKQYGNEVFLAKLIAQACVSIFPDSGHFNVDNIRVCKILGSGISSSSVLHGMVFKKETEGDVTSVKDAKIAVYSCPFDGMITETKGTVLIKTAEELMNFSKGEENLMDAQVKAIADTGANVVVTGGKVADMALHYANKYNIMLVRLNSKWDLRRLCKTVGATALPRLTPPVLEEMGHCDSVYLSEVGDTQVVVFKHEKEDGAISTIVLRGSTDNLMDDIERAVDDGVNTFKVLTRDKRLVPGGGATEIELAKQITSYG

In [51]:
seq_R, seq_L = get_sequences("5u9f__RA1_P0A7W1--5u9f__D1_P36675")


2025-03-27 18:52:46,887 | pinder.core.utils.cloud:375 | INFO : Gsutil process_many=download_to_filename, threads=7, items=7
2025-03-27 18:54:45,010 | pinder.core.utils.cloud.process_many:25 | ERROR : runtime failed:    118.12s


RetryError: Timeout of 120.0s exceeded, last exception: HTTPSConnectionPool(host='storage.googleapis.com', port=443): Max retries exceeded with url: /storage/v1/b/pinder/o/2024-02%2Fpdbs%2F5u9f__RA1_P0A7W1--5u9f__D1_P36675.pdb?projection=noAcl&prettyPrint=false (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 403 Forbidden')))

In [ ]:
print(seq_R)